In [2]:
search_for = 130
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.003993034362792969
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 749112551
type: [1, 1, 1, 1, 130] 130
cases of this type: 285610000
100000 33.96360637400794
200000 57.175973724363104
300000 53.59429887871258
400000 56.013071367554616
500000 54.36790863823128
600000 57.34099872897992
700000 55.09806619524432
800000 53.3917919886263
900000 55.42951706595457
1000000 52.902498789835946
1100000 39.66898766035657
1200000 32.36245184348823
1300000 52.39062914916198
1400000 54.58899806427978
1500000 51.81162723049739
1600000 54.65011608051481
1700000 55.47954765951111
1800000 53.19226757287309
1900000 54.82942971035753
2000000 52.85291727216987
2100000 55.280424550989196
2200000 40.434737715861125
2300000 27.140445814004504
2400000 54.62329822876346
2500000 51.31705849223026
2600000 54.202453076563465
2700000 52.692913602130695
2800000 54.35985412456785
2900000 53.157371582459184
3000000 50.8516309

29700000 12.032564467069719
29800000 29.409755411425746
29900000 22.895552160057242
30000000 32.74368618249213
30100000 41.35451208132734
30200000 47.955684763406246
30300000 53.37848095054485
30400000 47.953509234814604
30500000 53.87568291127562
30600000 22.918039716717377
30700000 50.45177726683901
30800000 52.55373472464657
30900000 45.718030361437116
31000000 41.17290219404587
31100000 35.36296840620153
31200000 50.48677444215533
31300000 53.163751063665465
31400000 45.228034854015085
31500000 51.06623152960314
31600000 44.99353542193304
31700000 46.430835413963635
31800000 46.04799070702055
31900000 18.745493507128117
32000000 50.54828678736139
32100000 42.302733827658656
32200000 48.86578083518349
32300000 44.6068151123826
32400000 47.264265715488094
32500000 52.09325215611728
32600000 45.60187372186206
32700000 47.93458066006759
32800000 30.728466385668582
32900000 46.40101381817001
33000000 52.40270532200324
33100000 43.99384562860249
33200000 46.73269869626994
33300000 31.031

59800000 28.544486920334148
59900000 40.4651287604662
60000000 43.12863854235652
60100000 43.29397538910111
60200000 46.31895205619248
60300000 38.35585331864354
60400000 43.43536042542159
60500000 23.495980186090776
60600000 43.4677119536732
60700000 42.975023025902146
60800000 40.93895347822304
60900000 46.74739872947627
61000000 43.70388291997141
61100000 24.73587410725377
61200000 47.73788219024828
61300000 44.64787727575115
61400000 53.32254922816664
61500000 43.12829472192052
61600000 55.92772271411737
61700000 55.705462212360146
61800000 55.1747787041259
61900000 59.75749693558249
62000000 33.54599073809509
62100000 39.81596490818242
62200000 53.53780938984856
62300000 50.33706568541676
62400000 53.098347335968185
62500000 48.70216636235958
62600000 45.98516517324158
62700000 26.410295821552353
62800000 50.02983816581647
62900000 51.600008115322524
63000000 49.19768474959028
63100000 54.5035508796478
63200000 52.44219024136575
63300000 21.79056628717099
63400000 55.8841517718767

89900000 47.89119816273935
90000000 49.51078007821738
90100000 50.906058007619315
90200000 51.67084042505791
90300000 51.79300847371729
90400000 48.44615859562791
90500000 49.89219327519683
90600000 47.5241281250888
90700000 46.38391404338288
90800000 25.279540065096175
90900000 40.2884229359342
91000000 47.00456849106416
91100000 47.06307709517222
91200000 24.7075275538411
91300000 39.18239967238444
91400000 46.40943706390843
91500000 45.936639281651864
91600000 30.03571441565943
91700000 37.31447245681307
91800000 46.982352601755224
91900000 47.1079058392606
92000000 49.545869097162075
92100000 49.70473817236074
92200000 50.35871991877981
92300000 51.85019223255819
92400000 52.09476634698512
92500000 54.14387835654016
92600000 49.97470306497874
92700000 49.81297747876977
92800000 47.787442666046104
92900000 46.92755592951279
93000000 30.41109025395651
93100000 33.54734602274245
93200000 47.291018816027496
93300000 47.045943761714206
93400000 24.621972646961428
93500000 41.71592241344

119300000 35.66847383662094
119400000 40.42428782095713
119500000 38.94754746086514
119600000 17.534307641154
119700000 42.17530686262125
119800000 12.650004683277546
119900000 40.06779714093541
120000000 23.4035926287126
120100000 38.62966945269649
120200000 41.48511483092087
120300000 35.27926609133302
120400000 42.080346439882845
120500000 37.35366744825637
120600000 39.73203785273746
120700000 43.429196527565196
120800000 40.37436584536367
120900000 45.134355713385865
121000000 40.634255172800955
121100000 39.79666498625455
121200000 39.84675267845068
121300000 35.26013882665496
121400000 41.59466063248416
121500000 34.7286793714587
121600000 39.90996057780546
121700000 38.02377054848782
121800000 20.94097449629012
121900000 40.977635323294784
122000000 12.559849051913892
122100000 34.62919328544385
122200000 27.883027921941032
122300000 37.56594514465901
122400000 41.1156358698947
122500000 34.20813952255734
122600000 40.98450951096042
122700000 37.01477181488454
122800000 38.2357

...

638400000 1.6501094658445075
638500000 1.5395067810119485
638600000 1.3436998403438074
638700000 1.3624178956917115
638800000 1.2072212043681063
638900000 1.0582511687483729
639000000 0.8785402363514113
639100000 1.8287528032423945
639200000 1.5299078646241746
639300000 1.3797722518162636
best so far: 0
type: [1, 1, 2, 13, 5] 130
cases of this type: 5492500
639400000 2.114469464892893
639500000 1.531710944145109
639600000 1.130239934311045
639700000 1.6294031532136275
639800000 1.5602126573510529
639900000 1.2491367267967584
640000000 0.9124482019216987
640100000 0.8894807295663377
640200000 1.0101598672095105
640300000 0.9712845607002828
640400000 1.2222750423159778
640500000 1.9114019942225833
640600000 1.6071139126257432
640700000 1.5398228150370121
640800000 1.4286455523090396
640900000 1.7333958607256272
641000000 1.5170006939642806
641100000 1.2253208856142805
641200000 1.562935086737291
641300000 0.9486022856894334
641400000 0.9277449992974136
641500000 0.9142257433427188
641600

665200000 1.4946931260545644
665300000 1.2800394620214064
665400000 0.5245885724958896
665500000 0.46015089064863784
665600000 0.35885903194947644
665700000 0.4221373799340983
665800000 0.4180550265503877
665900000 0.41743948938030606
666000000 0.45658098696324356
666100000 0.6091918406455987
666200000 0.434309221399961
666300000 0.4249667251204206
666400000 0.9474981908958117
666500000 1.2355322034678975
666600000 0.9944629235076621
666700000 1.0582628007433945
666800000 0.6925713394644498
666900000 0.8212120271928223
667000000 0.6412691068897015
667100000 0.9562404652815619
667200000 1.2643819266260048
best so far: 0
type: [1, 1, 10, 13, 1] 130
cases of this type: 219700
667300000 0.6319750048928539
667400000 0.30955823402995075
best so far: 0
type: [1, 1, 13, 1, 10] 130
cases of this type: 1690000
667500000 0.9576943326441916
667600000 2.2043054962243644
667700000 1.7545886157784578
667800000 1.7847239444112997
667900000 1.8846139026714046
668000000 1.3455316691821975
668100000 0.34

691000000 1.5184627586634993
691100000 1.5841234743317842
691200000 0.5023268151637766
691300000 1.1280676139610304
691400000 1.8599231887581846
691500000 2.1645952106375517
691600000 1.1917903823609315
691700000 1.543572909148832
691800000 1.083264133014112
691900000 1.845646320212723
692000000 1.2407841811445832
692100000 1.375884918957278
692200000 1.5219697268401893
692300000 1.0709046611843704
692400000 1.6984486671765844
692500000 1.6091274466920324
692600000 1.5757820049712437
692700000 1.5111580609418644
692800000 1.310420679384115
692900000 0.7179029119039892
693000000 1.2966929523054422
693100000 1.5729220449537673
693200000 1.1880280432443857
693300000 1.427782760301994
693400000 0.7422836508303172
693500000 1.1747465271580677
693600000 1.1355016279808408
693700000 1.2253024010383413
693800000 1.4956366948023319
693900000 0.9576260075015803
694000000 0.9572656360940093
694100000 1.4939540624602297
694200000 1.6727231866653756
694300000 1.2738671724567627
694400000 1.12342093

718300000 0.40948288466456734
718400000 0.3875512469265123
718500000 0.22408843695674008
718600000 0.24185783176757827
best so far: 0
type: [1, 2, 5, 13, 1] 130
cases of this type: 109850
718700000 0.2514496832219084
best so far: 0
type: [1, 2, 13, 1, 5] 130
cases of this type: 211250
718800000 0.11057854877174826
718900000 0.1240218325215631
719000000 0.12391941047480107
best so far: 0
type: [1, 2, 13, 5, 1] 130
cases of this type: 42250
best so far: 0
type: [1, 2, 65, 1, 1] 130
cases of this type: 8450
best so far: 0
type: [1, 5, 1, 1, 26] 130
cases of this type: 2284880
719100000 0.5015456611406227
719200000 1.0748708488161576
719300000 1.1085282573355992
719400000 0.9431837266228603
719500000 0.935399733032753
719600000 0.5576833806563297
719700000 0.6516887160737541
719800000 0.49542657652976513
719900000 0.4898180006281197
720000000 0.6093157479591013
720100000 0.29379125520032445
720200000 0.12046764128621222
720300000 0.11939828627061977
720400000 0.119253226030963
720500000 0.

738600000 0.24998557567784058
738700000 0.3097299763889445
738800000 0.2248651981913388
738900000 0.20701274317047066
739000000 0.1657765342006902
739100000 0.19465052890251278
739200000 0.32348436414931353
739300000 0.314045308542088
739400000 0.26948474132372535
739500000 0.21049888956681426
739600000 0.18471784495177865
739700000 0.25407317469540236
739800000 0.26366373092117507
739900000 0.22808178590629496
740000000 0.23418389076488813
740100000 0.24950306617225143
740200000 0.2275580611429314
740300000 0.1734207377111554
740400000 0.15274385931828882
740500000 0.1468490940810124
740600000 0.27027380589872596
740700000 0.1787602202831447
740800000 0.1534222114350564
740900000 0.25827341108837326
741000000 0.2749637877765066
741100000 0.16484601739231544
741200000 0.21572494744603435
741300000 0.18031079474613135
741400000 0.2128731164977405
741500000 0.1887514608561337
741600000 0.1550121874711825
741700000 0.29448696331855523
741800000 0.2043137628825585
best so far: 0
type: [2, 